In [ ]:
# ライブラリのインストール
!pip install flask flask-ngrok pyngrok flask-cors


from flask import Flask, request, send_file, jsonify, redirect
from flask_ngrok import run_with_ngrok
from pyngrok import ngrok
from flask_cors import CORS
import cv2
import numpy as np
import requests
from io import BytesIO
from PIL import Image
import subprocess
import time


# ngrokの認証トークンを設定
ngrok.set_auth_token("2m6oTjvBpHYvI2aGsdxwRsVmzeQ_6aDsCGnZDSpZcVrjq5vWq")


app = Flask(__name__)
# CORS(app, resources={r"/process_images": {"origins": ["http://127.0.0.1:5501"]}})
# 全てのオリジンからのリクエストを許可
CORS(app)


# GitHubから画像をダウンロードする関数
def download_image(url):
    url = url.replace("github.com", "raw.githubusercontent.com").replace("blob/", "")
    response = requests.get(url)
    print(f"Downloading image from: {url}, Status code: {response.status_code}")
    if response.status_code == 200:
        image = Image.open(BytesIO(response.content))
        return cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    else:
        raise FileNotFoundError(f"画像が見つかりません: {url}")


@app.route('/process_images', methods=['POST'])
def process_images():
    try:
        print(f"Request files: {request.files}")
        # 受け取ったファイルを取得
        if 'longshirt[]' not in request.files:
            print("No images provided in the request.画像が含まれていません。")
            return jsonify({"error": "No images provided."}), 400

        # longshirtのファイルをリストとして取得
        files = request.files.getlist('longshirt[]')  # 'longshirt[]'に変更
        print(f"Number of files received: {len(files)}")


        for file in files:
            print(f"File name: {file.filename}, Content type: {file.content_type}")


        if len(files) != 4:
            return jsonify({"error": "Invalid input, expected 4 images."}), 400





        image_points = [
            np.array([[155,172],[160,200],[140,215],[120,361],[269,363],[255,215],[230,199],[228,173],[206,160]]),
            np.array([[155,160],[156, 195],[138,211],[134,347],[257,349],[246,211],[233, 196],[235,160],[200,170]]),
            np.array([[137, 232],[144, 288],[152, 306],[243, 305],[253, 229],[227, 217],[176, 216]]),
            np.array([[137, 232],[152, 306],[243, 305],[253, 229],[227, 217],[176, 216]]),
        ]


        # 貼り付け先の画像のURL
        target_image_url = "https://ta03-jp.github.io/muchu3/bigT_frame_only.png"
        target_image = download_image(target_image_url)


        target_points_list = [
            np.array([[107, 40],[129, 118],[51, 180],[0, 647],[490, 651],[480, 180],[350, 118],[373, 41],[258, 0]]),
            np.array([[575, 0],[561, 133],[470, 180],[491, 649],[949, 651],[906, 182],[865, 113],[855, 0],[704, 57]]),
            np.array([[537, 736],[540, 915],[555, 967],[864, 967],[940, 743],[772, 620],[621, 620]]),
            np.array([[112, 743],[155, 967],[462, 967],[483, 737],[408, 640],[227, 620]]),
        ]


        # 画像を処理する
        for i, file in enumerate(files):
            image = Image.open(file.stream)
            image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)


            points = image_points[i]
            target_points = target_points_list[i]


            mask = np.zeros(image.shape[:2], dtype=np.uint8)
            cv2.fillPoly(mask, [points], 255)


            M, status = cv2.findHomography(points, target_points)


            transformed_image = cv2.warpPerspective(image, M, (target_image.shape[1], target_image.shape[0]))
            mask_transformed = cv2.warpPerspective(mask, M, (target_image.shape[1], target_image.shape[0]))
            target_image[mask_transformed > 0] = transformed_image[mask_transformed > 0]


        # 変更された画像を一時ファイルに保存
        modified_target_image_path = 'modified_image.jpg'
        cv2.imwrite(modified_target_image_path, target_image)


        # 生成された画像を返す
        return send_file(modified_target_image_path, mimetype='image/jpeg')



    except Exception as e:
          print(f"Error processing file {file.filename}: {e}")
          return jsonify({"error": f"Error processing file {file.filename}: {str(e)}"}), 500




# テスト用app
@app.route('/', methods=['GET'])
def home():
    return "Flask is running! APIテストOK"


# JSテスト用app
@app.route('/img', methods=['GET'])
def img():
    image_url = "https://github.com/ta03-jp/muchu3/blob/main/bigT_frame_only.png"
    return redirect(image_url)


if __name__ == '__main__':
    # ngrokを起動する
    ngrok_process = subprocess.Popen(['ngrok', 'http','--domain=sensible-trusted-hare.ngrok-free.app', '5000'])
    time.sleep(2)  # ngrokが立ち上がるまで待つ


    # ngrokのURLを取得
    url_info = requests.get('http://localhost:8080/api/tunnels').json()
    ngrok_url = url_info['tunnels'][0]['public_url']
    print(f"ngrok URL: {ngrok_url}")


    app.run(port=5000)


ngrok URL: https://sensible-trusted-hare.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [10/Dec/2024 08:07:24] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Dec/2024 08:07:24] "GET /favicon.ico HTTP/1.1" 404 -


Request files: ImmutableMultiDict([('longshirt[]', <FileStorage: 'image0.png' ('image/png')>), ('longshirt[]', <FileStorage: 'image1.png' ('image/png')>), ('longshirt[]', <FileStorage: 'image2.png' ('image/png')>), ('longshirt[]', <FileStorage: 'image3.png' ('image/png')>)])
Number of files received: 4
File name: image0.png, Content type: image/png
File name: image1.png, Content type: image/png
File name: image2.png, Content type: image/png
File name: image3.png, Content type: image/png


INFO:werkzeug:127.0.0.1 - - [10/Dec/2024 08:07:34] "POST /process_images HTTP/1.1" 200 -
